KABAHITA JUPITER MARINA
2019/HD07/24828U


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Importing data

In [ ]:
# Importing data
test_data = pd.read_csv("/kaggle/input/ace-class-assignment/Test.csv", header=0, sep=",")
amp_train = pd.read_csv("/kaggle/input/ace-class-assignment/AMP_TrainSet.csv", header=0, sep=",")
amp_train.head()





In [ ]:
test_data.head()

## General data attributes and descriptive statistics

In [ ]:
#Describing general data attributes
train_dim=amp_train.shape #gets the dimensions of the tranining dataset
test_dim= test_data.shape #gets the dimensions of the tesing dataset
print(train_dim)
print(test_dim)



We can see that the test data has one attribute less than the train data set

In [ ]:
atr_types=amp_train.dtypes #gets the data type for each column in training dataset

atr_types



In [ ]:
amp_train.isnull().sum() # gets the sum of the missing values/observations in each column

 All attributes have complete observations

In [ ]:
# Descriptive statistics 
stats=amp_train.describe()
stats

From the summary statistics, its visible that some of the attributes like FULL_CHARGE, FULL_AcidicMolPerc and	FULL_OOBM850104  have outlier values basing on the values of the 75% quartile and maximum values 

# Class distributions

Sometimes, observations from the different classes may not be equal, these may need special handling, so its important to note the class distributions, as if they are uneven, they may bias our model

In [ ]:
# CLass distibutions 
cdists= amp_train.groupby('CLASS').size() # groups the observation by the column of class, and counts all the observations including null values
print(cdists)

#plots to visualize the class distribution
amp_train.groupby('CLASS').size().plot(kind="pie")

The observations from each class seem8 to be equal

# Correlation between the different attributes
Basically, we need to examine our attributes for correlation as highly correlated attributes present the same data to the algorithm. Removing one may speed learning of the algorithm (less dimensions more speed) and also prevent overfitting of the algorithm

In [ ]:
# Correlation between attributes 
correlations = amp_train.corr(method="pearson") # calculates pairwaise correlation for the attributes 
correlations



FULL_AURR980107 and FULL_AcidicMolPerc have a correlation coefficent of 0.79, this might mean these two attributes may be some what correlated. Same goes for AS_DAYM780201 and FULL_DAYM780201 (correlation coefficient is 0.894191)

In [ ]:
# Plots to visualize the correlations
from matplotlib import rcParams
sns.heatmap(correlations, annot=True)
rcParams['figure.figsize'] = (11.7,8.27)


# Data skewness

Most machine learning algorithms assume that data from the attributes is normally distributed, so identifying attributes with data that has a left or right skew, can be important as this can be easily corrected

In [ ]:
sk = amp_train.skew() # calculates the skew for each attribute
sk

Negative values like those of FULL_DAYM780201, FULL_OOBM850104,AS_DAYM780201, AS_FUKS010112, indicate that these attributes have values skewed more to the left
Values closer to zero are indicative of a less or no skew, while positive values would mean that that particular attribute is skewed more to the right

 # Visualising data distributions with plots

In [ ]:
# Plot one, histogram to show the distributions
amp_train.hist(layout=(4,3), figsize=(16,16))
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.show()

In [ ]:
# Plot two density plots can show the distributions better
amp_train.plot(kind="density",subplots=True,layout=(4,3),sharex=False,sharey=False, figsize=(16,16))
#amp_train.plot(kind="density",subplots=False,layout=(4,3),sharex=True,sharey=True)
plt.show()


Distributions for AS_MeanAmphiMoment,FULL_AcidicMolPerc and NT_EFC195 dont look so good

# Data transformations
Using squaring to transform the attributes with negative values into postives

In [ ]:

neg1=amp_train.iloc[:,0]
tneg1=neg1**2
tneg1.plot(kind="density")

amp_train.iloc[:,0]=tneg1

neg2=amp_train.iloc[:,5]
tneg2=neg2**2
tneg2.plot(kind="density")
amp_train.iloc[:,5]=tneg2

## test
neg3=test_data.iloc[:,0]
tneg3=neg3**2
tneg3.plot(kind="density")
test_data.iloc[:,0]=tneg3



neg4=test_data.iloc[:,5]
tneg4=neg4**2
tneg4.plot(kind="density")
test_data.iloc[:,5]=tneg4

# Feature selection




In [ ]:
 #test one selecting out features with low variance
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
k=sel.fit_transform(amp_train)
names=amp_train.columns[sel.get_support(indices=True)]
print(k.shape)
amp_train2=pd.DataFrame(k,columns=names)
amp_train2.head()

After selecting out features with low variance, only 8 attribute remained 

In [ ]:
#test two using sklearn with a chi square test

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test_array = amp_train2.values

test_atr = test_array[:,0:7]
class_atr = test_array[:,7]


test = SelectKBest(score_func=chi2, k=5)
fit = test.fit(test_atr, class_atr)


print(fit.scores_)
features = fit.transform(test_atr)
names2=amp_train2.columns[fit.get_support(indices=True)]
amp_train3=pd.DataFrame(features,columns=names2)
amp_train3.iloc[:5,]
print(names2)





In [ ]:
# Test three Recursive feature elimination

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier

model1 = LogisticRegression()
model2=RandomForestClassifier()

rfe = RFE(model1, 5)
fit2 = rfe.fit(test_atr, class_atr)
features2 = fit2.transform(test_atr)

rfe2= RFE(model2, 5)
fit3 = rfe2.fit(test_atr, class_atr)




names3= set(amp_train2.columns[fit2.get_support(indices=True)])

names4= set(amp_train2.columns[fit3.get_support(indices=True)])

names2=set(names2)

comn=names2&names3&names4 ### identifies features common to all the feature selection methods used
comn

import collections

comn=collections.deque(comn)
comn.appendleft("FULL_Charge")
comn



Getting the indices of the chosen attributes

In [ ]:

indice=[]

for index,name in enumerate(amp_train.columns):
    for sub_name in comn:
        if name==sub_name:
            indice.append(index)
indice
            

Subsetting the data for the chosen attributes 

In [ ]:
amp_train = pd.read_csv("/kaggle/input/ace-class-assignment/AMP_TrainSet.csv", header=0, sep=",")


test_amp =amp_train.iloc[:,[0,1,3,5,7,11]]

test_amp.head()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


## Logistic regression using cross validation

In [ ]:

X=test_amp.iloc[:,0:5]
Y=test_amp.iloc[:,5]
folds=10
kfold = KFold(n_splits=folds, random_state=7, shuffle=True,)
model = LogisticRegression()
results = cross_val_score(model, X, Y, cv=kfold)

print(("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0))





# Model one
 ## Logistic regression with a test train split using 5 features
 

In [ ]:
test_amp =amp_train.values[:,[0,1,3,5,7,11]]
from sklearn.metrics import matthews_corrcoef
X=test_amp[:,0:5]
Y=test_amp[:,5]

#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model1 = LogisticRegression()
model1.fit(X_train, Y_train)

# predicting

predicted = model1.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result1 = model1.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result1*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model1.predict(X_test),Y_test))
mcc1=matthews_corrcoef(model1.predict(X_test),Y_test)


 # Model two
## Gausian/Naive Bayes using five features 

In [ ]:
test_amp =amp_train.values[:,[0,1,3,5,7,11]]
from sklearn.metrics import matthews_corrcoef
X=test_amp[:,0:5]
Y=test_amp[:,5]

#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model2 = GaussianNB()
model2.fit(X_train, Y_train)

# predicting

predicted = model2.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result2 = model2.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result2*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model2.predict(X_test),Y_test))
mcc2=matthews_corrcoef(model2.predict(X_test),Y_test)

# Model three
## Linear Discriminant Analysis using five features 

In [ ]:
test_amp =amp_train.values[:,[0,1,3,5,7,11]]
from sklearn.metrics import matthews_corrcoef
X=test_amp[:,0:5]
Y=test_amp[:,5]

#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model3=LinearDiscriminantAnalysis()
model3.fit(X_train, Y_train)

# predicting

predicted = model3.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result3 = model3.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result3*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model3.predict(X_test),Y_test))

mcc3=matthews_corrcoef(model3.predict(X_test),Y_test)



# Model four 
 ## Support Vector Machines with five features

In [ ]:
test_amp =amp_train.values[:,[0,1,3,5,7,11]]
from sklearn.metrics import matthews_corrcoef
X=test_amp[:,0:5]
Y=test_amp[:,5]

#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model4=SVC()
model4.fit(X_train, Y_train)

# predicting

predicted = model4.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result4 = model4.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result4*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model4.predict(X_test),Y_test))


mcc4=matthews_corrcoef(model4.predict(X_test),Y_test)



# model 5
 ## K-Nearest neighbours with five features
 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model5 = KNeighborsClassifier(n_neighbors=5)
#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model5.fit(X_train, Y_train)

# predicting

predicted = model5.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result5 = model5.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result5*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model5.predict(X_test),Y_test))

mcc5=matthews_corrcoef(model5.predict(X_test),Y_test)



# Model six
 ## Centroid clasifier 

In [ ]:
from sklearn.neighbors import NearestCentroid
model6 = NearestCentroid()
#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model6.fit(X_train, Y_train)

# predicting

predicted = model6.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result6 = model6.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result6*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model6.predict(X_test),Y_test))

mcc6=matthews_corrcoef(model6.predict(X_test),Y_test)



# Model seven

## Decision trees with five features 


In [ ]:
from sklearn import tree
#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model7=tree.DecisionTreeClassifier()
model7.fit(X_train, Y_train)

# predicting

predicted = model7.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result7 = model7.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result7*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model7.predict(X_test),Y_test))
mcc7=matthews_corrcoef(model7.predict(X_test),Y_test)


# Model eight 
## Randomised tree

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model8 =RandomForestClassifier()
#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model8.fit(X_train, Y_train)

# predicting

predicted = model8.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result8 = model8.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result8*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model8.predict(X_test),Y_test))


mcc8=matthews_corrcoef(model8.predict(X_test),Y_test)

# Model nine 
 ## Gradient tree boosting


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model9 =GradientBoostingClassifier()

#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model9.fit(X_train, Y_train)

# predicting

predicted = model9.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result9 = model9.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result9*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model9.predict(X_test),Y_test))

mcc9=matthews_corrcoef(model9.predict(X_test),Y_test)



# Model ten
## Stochastic Gradient Descent 

In [ ]:
from sklearn.linear_model import SGDClassifier
model10 = SGDClassifier()

#Training data spliting 

test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)

#model training 

model10.fit(X_train, Y_train)

# predicting

predicted = model10.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result10 = model10.score(X_test, Y_test)

# Results accuracy 

print("Accuracy: ",  (result10*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model10.predict(X_test),Y_test))


mcc10=matthews_corrcoef(model10.predict(X_test),Y_test)


In [ ]:
compare=[result1,result2,result3,result4,result5,result6,result7,result8,result9,result10]

compare=pd.DataFrame(compare).T

compare=compare.rename(columns={0:"LGR", 1:"GNB", 2:"LDN", 3:"SVM", 4:"KNN", 5:"CCL", 6:"DCS", 7:"RDT", 8:"GBT", 9:"SGD"})

compareMCC=[mcc1,mcc2,mcc3,mcc4,mcc5,mcc6,mcc7,mcc8,mcc9,mcc10]
compareMCC=pd.DataFrame(compareMCC).T
compareMCC=compareMCC.rename(columns={0:"LGR", 1:"GNB", 2:"LDN", 3:"SVM", 4:"KNN", 5:"CCL", 6:"DCS", 7:"RDT", 8:"GBT", 9:"SGD"})

compare=compare.append(compareMCC)
#compare.plot(kind="box")
#plt.plot("Mean of MCC and Accuracy","Different models", data=compare)


compare=compare.values

fig = plt.figure()
ax = fig.add_subplot(111)
ax.boxplot(compare)
plt.xticks([1, 2, 3,4,5,6,7,8,9,10], ["LGR","GNB","LDN","SVM","KNN","CCL","DCS","RDT","GBT","SGD"])



ax.set_title('A comparison of the different Accuracies and MCC from the different Models')
ax.set_xlabel('The different Models')
ax.set_ylabel('Mean of Accuracy and MCC')
plt.show()




# Predicting from the test data

In [ ]:

test_data = pd.read_csv("/kaggle/input/ace-class-assignment/Test.csv", header=0, sep=",")
test_data2=test_data.values[:,[0,1,3,5,7]]

### Predictions from logistic Regression

In [ ]:
model1.fit(X,Y) ## retrains the algorithm again on the entire train data set
kj_results=model1.predict(test_data2)

df = pd.DataFrame(kj_results)
df.columns = ['CLASS']
df.index.name = 'Index'
df['CLASS']=df['CLASS'].map({0.0:False,1.0:True})
df.to_csv("kjLGR.csv")
df["CLASS"].unique()
print(df.groupby("CLASS").size()[0].sum())
print(df.groupby("CLASS").size()[1].sum())
df['CLASS'].value_counts()


Score from the leader board using this prediction is 0.82503

## Predicting using Gausssian

In [ ]:
model2.fit(X,Y) ## retrains the algorithm again on the entire train data set
kj_results=model2.predict(test_data2)

df = pd.DataFrame(kj_results)
df.columns = ['CLASS']
df.index.name = 'Index'
df['CLASS']=df['CLASS'].map({0.0:False,1.0:True})
df.to_csv("kjGNB.csv")
df["CLASS"].unique()
print(df.groupby("CLASS").size()[0].sum())
print(df.groupby("CLASS").size()[1].sum())
df['CLASS'].value_counts()

Score on leader board using this prediction was 0.84591

# Predicting using Support Vector Machines  

In [ ]:
model4.fit(X,Y) ## retrains the algorithm again on the entire train data set
kj_results=model4.predict(test_data2)

df = pd.DataFrame(kj_results)
df.columns = ['CLASS']
df.index.name = 'Index'
df['CLASS']=df['CLASS'].map({0.0:False,1.0:True})
df.to_csv("kjSVM.csv")
df["CLASS"].unique()
print(df.groupby("CLASS").size()[0].sum())
print(df.groupby("CLASS").size()[1].sum())
df['CLASS'].value_counts()

Score on leader board using this prediction was 0.82136

## Predicting using the randomised tree classifier

In [ ]:
model8.fit(X,Y) ## retrains the algorithm again on the entire train data set
kj_results=model8.predict(test_data2)

df = pd.DataFrame(kj_results)
df.columns = ['CLASS']
df.index.name = 'Index'
df['CLASS']=df['CLASS'].map({0.0:False,1.0:True})
df.to_csv("kjRDT.csv")
df["CLASS"].unique()
print(df.groupby("CLASS").size()[0].sum())
print(df.groupby("CLASS").size()[1].sum())
df['CLASS'].value_counts()

Score on leader board using this prediction was 0.80561

## Predicting using the Gradient tree boosting classifier 

In [ ]:
model9.fit(X,Y) ## retrains the algorithm again on the entire train data set
kj_results=model9.predict(test_data2)

df = pd.DataFrame(kj_results)
df.columns = ['CLASS']
df.index.name = 'Index'
df['CLASS']=df['CLASS'].map({0.0:False,1.0:True})
df.to_csv("kjGBT.csv")
df["CLASS"].unique()
print(df.groupby("CLASS").size()[0].sum())
print(df.groupby("CLASS").size()[1].sum())
df['CLASS'].value_counts()

Score on leader board using this prediction was 0.79678

Random forest and Gradient boosting classifiers had the highest accuracies during training but gave the least scores on the leader board

Gausian gave the highest score

# More with the gausian 

## Using gausian with more features

## Using gausian with eight features 

In [ ]:
amp_train = pd.read_csv("/kaggle/input/ace-class-assignment/AMP_TrainSet.csv", header=0, sep=",")
test_data = pd.read_csv("/kaggle/input/ace-class-assignment/Test.csv", header=0, sep=",")

#selecting out features with low variance
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
k=sel.fit_transform(amp_train)
names=amp_train.columns[sel.get_support(indices=True)]
print(k.shape)
amp_train2=pd.DataFrame(k,columns=names)
#print(amp_train2)

n=sel.fit_transform(test_data)
names2=test_data.columns[sel.get_support(indices=True)]
test_data=pd.DataFrame(n,columns=names2)
test_data2=test_data.values

#print(test_data)

### data 
test_amp =amp_train2.values
X=test_amp[:,0:7]
Y=test_amp[:,7]




#model
model=GaussianNB()
from sklearn.metrics import matthews_corrcoef
test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,
random_state=seed, shuffle="True")
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result = model.score(X_test, Y_test)
print("Accuracy: ",  (result*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model.predict(X_test),Y_test))  
model.fit(X,Y)
kj_resultsg8=model.predict(test_data)


#### results 

df = pd.DataFrame(kj_resultsg8)
df.columns = ['CLASS']
df.index.name = 'Index'
df['CLASS']=df['CLASS'].map({0.0:False,1.0:True})

df.to_csv("kjGNB8.csv")
df["CLASS"].unique()
print(df.groupby("CLASS").size()[0].sum())
print(df.groupby("CLASS").size()[1].sum())
df['CLASS'].value_counts()


MCC value for the Gaussian with 8 features and that with 5 features are almost the same, but the leaderboard score using the gausian with 8 features was 0.86379, which is an improvement over the 0.84 score from the five features

## Using gaussian with all attributes 

In [ ]:
amp_train = pd.read_csv("/kaggle/input/ace-class-assignment/AMP_TrainSet.csv", header=0, sep=",")
test_data = pd.read_csv("/kaggle/input/ace-class-assignment/Test.csv", header=0, sep=",")


### data 
test_amp =amp_train.values
X=test_amp[:,0:11]
Y=test_amp[:,11]


#model
model=GaussianNB()
from sklearn.metrics import matthews_corrcoef
test_size = 0.4
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,
random_state=seed)
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result = model.score(X_test, Y_test)
print("Accuracy: ",  (result*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model.predict(X_test),Y_test))  
model.fit(X,Y)

kj_resultsg12=model.predict(test_data)


#### results 

df = pd.DataFrame(kj_resultsg12)
df.columns = ['CLASS']
df.index.name = 'Index'
df['CLASS']=df['CLASS'].map({0.0:False,1.0:True})

df.to_csv("kjGNB11.csv")
df["CLASS"].unique()
print(df.groupby("CLASS").size()[0].sum())
print(df.groupby("CLASS").size()[1].sum())
df['CLASS'].value_counts()



Score from the leaderboard using all features was 0.99559

References


1. https://realpython.com/logistic-regression-python/
2. https://www.tutorialspoint.com/machine_learning_with_python/machine_learning_with_python_extra_trees.htm
3. https://stackabuse.com/gradient-boosting-classifiers-in-python-with-scikit-learn/
4. https://machinelearningmastery.com/stochastic-gradient-boosting-xgboost-scikit-learn-python/
5. https://machinelearningmastery.com/stochastic-gradient-boosting-xgboost-scikit-learn-python/
6. https://machinelearningmastery.com/compare-machine-learning-algorithms-python-scikit-learn/
7. https://machinelearningmastery.com/master-machine-learning-algorithms/
8. https://machinelearningmastery.com/compare-machine-learning-algorithms-python-scikit-learn/
9. https://github.com/search?q=data-analysis-and-visualization-with-python&type=Repositories
10. https://stackabuse.com/applying-filter-methods-in-python-for-feature-selection/
11. https://towardsdatascience.com/decision-tree-in-machine-learning-e380942a4c96
12. https://machinelearningmastery.com/master-machine-learning-algorithms/
13. https://machinelearningmastery.com/compare-machine-learning-algorithms-python-scikit-learn/

